<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Задание-1" data-toc-modified-id="Задание-1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Задание 1</a></span></li><li><span><a href="#Задание-2" data-toc-modified-id="Задание-2-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Задание 2</a></span></li><li><span><a href="#Задание-3-*" data-toc-modified-id="Задание-3-*-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Задание 3 *</a></span><ul class="toc-item"><li><span><a href="#Список-объектов" data-toc-modified-id="Список-объектов-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Список объектов</a></span></li></ul></li><li><span><a href="#Данные-по-определенному-объекту" data-toc-modified-id="Данные-по-определенному-объекту-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Данные по определенному объекту</a></span></li></ul></div>

# Задание 1
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание 2
* Выберите регион
* Сравните среднюю цену квадратного метра в городах этого региона
* Определите количество строящихся объектов и квартир в регионах
* Определите объем строящегося жилья в квадратных метрах

# Задание 3 *
* Напишете скрипт, который сможет ежедневно собирать данные по цене, количестве и объему строящихся объектов самостоятельно и ежедневно
* Попробуйте ускорить процесс
* Добавьте возможность добавления данных в БД с указанием текущей даты.

In [47]:
import requests
from tqdm.auto import tqdm
import json

## Список объектов

In [48]:
url_0 = url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset=0&limit=0&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res_0 = requests.get(url_0)
maximum = res_0.json()
t = list(maximum['data'].items())
t2 = list(t[1])
total = t2[1]
total #количество строящихся объектов на сайте

10018

In [49]:
from tqdm.auto import tqdm
limit_ = 1000
all_obj_ids = []
for offset_ in tqdm(range(0,12000,1000)):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)
    objects_data = res.json()
    objects_list = objects_data.get('data').get('list')
    objids = [x.get('objId') for x in objects_list]
    all_obj_ids.extend(objids)

  0%|          | 0/1 [00:00<?, ?it/s]

In [50]:
len(all_obj_ids)

500

# Данные по определенному объекту

In [51]:
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/45434'

In [52]:
res = requests.get(url)
#res.json()

In [53]:
#Задание 1

In [54]:
import pandas as pd

In [55]:
df = pd.DataFrame()
for i in tqdm(all_obj_ids):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{i}'
    res = requests.get(url)
    df = pd.concat([df, pd.json_normalize(res.json())], axis=0)

  0%|          | 0/500 [00:00<?, ?it/s]

In [56]:
#Cохранить pandas dataframe в excel

In [57]:
writer = pd.ExcelWriter('our_house_base.xlsx')
df.to_excel(writer)
writer.save()
writer.close()

In [58]:
#Cохранить pandas dataframe в БД

In [59]:
from sqlalchemy import create_engine
import sqlite3

In [60]:
engine = create_engine('sqlite:///our_house_db.db', echo=True)
sqlite_connection = engine.connect()

In [61]:
sqlite_table = 'our_house_baza_'
df.to_sql(sqlite_table, sqlite_connection, if_exists='fail')
sqlite_connection.close()

In [62]:
sqlite3

In [63]:
#Cохранить pandas dataframe в pickle

In [64]:
import pickle

In [65]:
pd.to_pickle(df,'./our_house_base_pik.txt')

In [66]:
df_pickl = pd.read_pickle('./our_house_base_pik.txt')

In [67]:
df_pickl.head()

In [68]:
#Задание 2

In [69]:
#Выберите регион
#Сравните среднюю цену квадратного метра в городах этого региона

In [70]:
mos_region = 50

In [71]:
df_mos_regs = df[df['data.region'] == mos_region]

In [72]:
df_f = df_mos_regs['data.address'].str.split(',', expand=True)
df_mos_regs['city'] = df_f[0]
df_mos_regs['city'] = df_f

<ipython-input-72-c7a73fababcc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mos_regs['city'] = df_f[0]
<ipython-input-72-c7a73fababcc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mos_regs['city'] = df_f


In [73]:
df_mos_regs_avg = df_mos_regs.groupby(['city'])['data.objPriceAvg'].mean().sort_values(ascending=False)

In [74]:
df_mos_regs_avg

city
д Сапроново    184017.0
г Лыткарино     82256.0
г Дмитров       74893.0
г Лобня         68765.0
Name: data.objPriceAvg, dtype: float64

In [75]:
#Определите количество строящихся объектов и квартир в регионах

In [81]:
count_obj = df.groupby('data.region')['data.id'].count()
count_obj #количество строящихся объектов в регионах

data.region
1       8
2       2
5       1
15      9
16      4
18      3
22      1
23     32
25      3
26      1
27      1
28      1
32      1
33      5
34      3
35      1
39      7
40      4
44      2
45      1
47     42
50      5
52      5
54     77
55      1
57      1
59      1
60      2
61      1
62      4
63      8
66     11
69      1
71     12
72     17
76      3
77    151
78     61
86      7
Name: data.id, dtype: int64

In [82]:
count_obj_elem = df.groupby('data.region')['data.objFlatCnt'].sum()
count_obj_elem #количество строящихся квартир в регионах

data.region
1       785
2       545
5       440
15      764
16     1956
18      564
22        0
23     7318
25      296
26      104
27      116
28       98
32      144
33     1002
34      209
35      223
39     1328
40      659
44       60
45      348
47     3177
50     1714
52      658
54     4002
55        0
57      349
59      321
60      232
61        0
62      823
63     1362
66     1819
69       50
71      841
72     2853
76      197
77    15799
78     7761
86      709
Name: data.objFlatCnt, dtype: int64

In [78]:
#Определите объем строящегося жилья в квадратных метрах

In [141]:
df_vol = df['data.objFlatSq'].astype(float).sum()

In [142]:
df_vol #объем строящегося жилья в квадратных метрах

3271986.42